# Using the Deployed agent

Now that we've got it deployed and hosted, let's see how we can interface with it through their python SDK

There is also, of course, [a more involved API](https://langchain-ai.github.io/langgraph/cloud/reference/api/api_ref.html) for production ready environments.

In [1]:
from langgraph_sdk import get_client

client = get_client(url="https://byo-chatgpt-4489e8808ae65dc39c634b79a9031f9e-ffoprvkqsa-uc.a.run.app")
# create thread
thread = await client.threads.create()
print(thread)

{'thread_id': 'bec69f2b-e648-4837-9a49-ecb23314afd5', 'created_at': '2024-07-12T20:53:09.838222+00:00', 'updated_at': '2024-07-12T20:53:09.838222+00:00', 'metadata': {}, 'status': 'idle'}


In [2]:
input = {
    "messages": [
        {
            "role": "human",
            "content": "can you make me a picture of the rain"
        }
    ]
}
chunks = []
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=input,
    stream_mode="updates",
):
    print(f"Receiving new event of type: {chunk.event}...")
    print(chunk.data)
    chunks.append(chunk.data)
    print("\n\n")

Receiving new event of type: metadata...
{'run_id': '1ef4090c-6b66-68dd-8b79-ebc591de23e2'}



Receiving new event of type: updates...
{'agent': {'messages': [{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_dN5xxh7zOZ7XRtcxpzzG2jfY', 'function': {'arguments': '{"prompt":"a surreal and artistic representation of the rain"}', 'name': 'generate_dalle_image'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, 'type': 'ai', 'name': None, 'id': 'run-34a7d1fa-5bae-41a9-93e5-d81b40775bb3', 'example': False, 'tool_calls': [{'name': 'generate_dalle_image', 'args': {'prompt': 'a surreal and artistic representation of the rain'}, 'id': 'call_dN5xxh7zOZ7XRtcxpzzG2jfY'}], 'invalid_tool_calls': [], 'usage_metadata': None}]}}



Receiving new event of type: updates...
{'tools': {'messages': [{'content': 'https://oaidalleapiprodscus.blob.core.windows.net/private/org-V9dfKsKmgBG4voYBmvfUac3K/user-GXGJJ8NRlcHUZkh

---
# Quick Interface Example

Using the python SDK and gradio, making a super simple quick chatbot interface

In [3]:
import gradio as gr
import asyncio
from langgraph_sdk import get_client
import os

URL = "https://byo-chatgpt-4489e8808ae65dc39c634b79a9031f9e-ffoprvkqsa-uc.a.run.app"
API_KEY = os.environ["LANGSMITH_API_KEY"]

async def setup():
    client = get_client(url=URL, api_key=API_KEY)
    assistants = await client.assistants.search()
    assistant = assistants[0]
    thread = await client.threads.create()
    return assistant["assistant_id"], thread['thread_id']

assistant_id = None
thread_id = None

async def chat(message):
    global assistant_id, thread_id
    
    if assistant_id is None or thread_id is None:
        assistant_id, thread_id = await setup()
    
    client = get_client(url=URL, api_key=API_KEY)
    input_data = {"messages": [{"role": "user", "content": message}]}
    
    full_response = ""
    tool_calls = []
    async for chunk in client.runs.stream(
        thread_id,
        assistant_id,
        input=input_data,
        stream_mode="updates",
    ):
        if chunk.data and chunk.event != "metadata":
            if 'messages' in chunk.data.get('agent', {}):
                for msg in chunk.data['agent']['messages']:
                    if msg.get('type') == 'ai':
                        tool_calls.extend(extract_tool_calls(msg))
                        full_response += msg.get('content', '')

    return tool_calls, full_response if full_response else "No response generated."

def extract_tool_calls(message):
    calls = []
    if 'tool_calls' in message:
        for call in message['tool_calls']:
            call_description = f"Tool call: {call['name']} with args: {call['args']}"
            calls.append(call_description)
    return calls

def user(user_message, history):
    return "", history + [[user_message, None]]

def bot(history):
    user_message = history[-1][0]
    tool_calls, bot_message = asyncio.run(chat(user_message))
    
    new_history = list(history)
    if tool_calls:
        tool_call_text = "\n".join(tool_calls)
        new_history.append((None, tool_call_text))
    
    new_history.append((None, bot_message))
    
    return new_history

with gr.Blocks(theme=gr.themes.Base()) as demo:
    chatbot = gr.Chatbot(bubble_full_width=False)
    msg = gr.Textbox()
    submit = gr.Button("Submit")
    clear = gr.Button("Clear")

    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(share=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
demo.close()

Closing server running on port: 7860
